<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import io
import re
import os
import PyPDF2
import pandas as pd
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [3]:
file_objects = files.upload()

Saving Extrato XP - Cliente 01.pdf to Extrato XP - Cliente 01 (3).pdf


In [4]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

def remove_header_and_footer(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

def create_new_df():

    # Cria um DataFrame utilizado no consolidado final
    return pd.DataFrame({
        'CLASSIFICAÇÃO': pd.Series(dtype='str'),
        'ATIVO': pd.Series(dtype='str'),
        'EXPOSIÇÃO': pd.Series(dtype='str'),
        'TIT/INST': pd.Series(dtype='str'),
        'ATUAL': pd.Series(dtype='str'),
        'DATA': pd.Series(dtype='str'),
        'ON/OFF': pd.Series(dtype='str'),
        'DATA DE VENCIMENTO': pd.Series(dtype='str')
    })

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}\d{1,2}\b', ativo)
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações"
        ],
        "Renda Fixa": [
            "Renda Fixa","CDB","LC","LCA","LCI","CRA","CRI","Deb","Debenture",
            "Debênture","Tesouro","Pré","Pós","IPCA","IPCA+","LTN","NTN","NTN",
            "NTN-B","NTNB","DI","REF","DI","CP","Crédito Privado","LP","PGBL"
            "Longo Prazo","Providencia","Providência","Prev","VGBL"
        ], 
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","Multiestratégia","COE"
        ]
    }

    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo)
            if founded:
                return classificacao


    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo)
    if founded:
        return "Ações"

    return ""

In [5]:
text_body = ""
text_miner = ""

for file in file_objects.keys():

    file_name, file_extension = os.path.splitext(file)

    if file_extension != '.pdf':
        continue

    # Lê o conteúdo do arquivo PDF
    text_miner = convert_pdf_to_txt(file)
    
    reader = PyPDF2.PdfReader(file)

    for page in reader.pages:

        parts = []

        page.extract_text(visitor_text=remove_header_and_footer)
        text_body = text_body + "".join(parts)

In [6]:
text_miner

'Extrato de Cotista\n\nConsolidado\n\nMovimentação de 15/05/2022 a 11/11/2022\n\nNome do Cliente\nEndereço\n\nCEP: 02440-00\n\nPOSIÇÃO CONSOLIDADA\n\nFundo\n\nCota\n\nQuantidade\n\nValor Aplicado\n\nValor Bruto\n\nIR\n\nSulAmérica Inflatie FI Renda Fixa Longo Prazo\n\nXP Referenciado FI Referenciado DI CP\n\nXPA MM II FIC FIM\n\nXPA RV FIC FIM\n\nXPA Vértice Crédito FIM CP\n\n49,03907260\n\n2,55168782\n\n1,48495549\n\n1,51471586\n\n1,31704969\n\n485,46020696\n\n9.607,86926679\n\n206.015,91600648\n\n86.244,99799744\n\n62.558,82325043\n\n21.434,59\n\n23.303,62\n\n231.165,15\n\n119.600,00\n\n67.664,32\n\n23.806,52\n\n24.516,28\n\n305.924,47\n\n130.636,67\n\n82.393,08\n\n13,80\n\n272,84\n\n2.095,63\n\n1.655,50\n\n816,74\n\nEmissão: 11/11/2022\n\nCódigo: 3Nxxx7\n\nIOF\n\n0,00\n\n0,00\n\n0,00\n\n0,00\n\n0,00\n\nValor Liquido\n\n23.792,72\n\n24.243,44\n\n303.828,84\n\n128.981,17\n\n81.576,34\n\nTotal na Instituição\n\n463.167,68\n\n567.277,02\n\n4.854,51\n\n0,00\n\n562.422,51\n\nSulAmérica In

In [7]:
# Regex para obter a data de emissão
regex_0 = "(?<=Emiss.o: )\d{2}\/\d{2}\/\d{4}"
data_emissao = str(re.search(regex_0, text_miner, flags=(re.IGNORECASE)).group(0))

# Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
regex_1 = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
result_1 = str(re.search(regex_1, text_miner, flags=(re.IGNORECASE | re.DOTALL)).group(0))

# Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
regex_2 = "^.*[a-zA-Z] .*$"
result_2 = re.findall(regex_2, result_1, flags=(re.MULTILINE))

# Gera um dicionário de termos com espaço
dictionary = {}
print("TEXTOS A SEREM CORRIGIDOS:")
for result in result_2:
    dictionary[result.replace(" ","")] = result
    print(" - " + result)

TEXTOS A SEREM CORRIGIDOS:
 - Valor Aplicado
 - Valor Bruto
 - SulAmérica Inflatie FI Renda Fixa Longo Prazo
 - XP Referenciado FI Referenciado DI CP
 - XPA MM II FIC FIM
 - XPA RV FIC FIM
 - XPA Vértice Crédito FIM CP


In [8]:
# Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
regex_3 = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
result_3 = str(re.search(regex_3, text_body, flags=(re.IGNORECASE | re.DOTALL)).group(0))

# Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
result_3 = result_3.replace(" ",";")
print(result_3)

Fundo;Cota;Quantidade;ValorAplicado;ValorBruto;IR;IOF;ValorLiquido
SulAméricaInflatieFIRendaFixaLongoPrazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XPReferenciadoFIReferenciadoDICP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPAMMIIFICFIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPARVFICFIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPAVérticeCréditoFIMCP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [9]:
# Altera todos os textos incorretos para a versão correta
for key in dictionary:
    if key in result_3:
        result_3 = result_3.replace(key,dictionary[key])

print(result_3)

Fundo;Cota;Quantidade;Valor Aplicado;Valor Bruto;IR;IOF;ValorLiquido
SulAmérica Inflatie FI Renda Fixa Longo Prazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XP Referenciado FI Referenciado DI CP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPA MM II FIC FIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPA RV FIC FIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPA Vértice Crédito FIM CP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [10]:
# Converte o CSV para o tipo DataFrame
df = pd.read_csv(StringIO(result_3), sep=";")
df

,Fundo,Cota,Quantidade,Valor Aplicado,Valor Bruto,IR,IOF,ValorLiquido
0,SulAmérica Inflatie FI Renda Fixa Longo Prazo,"49,03907260","485,46020696","21.434,59","23.806,52","13,80","0,00","23.792,72"
1,XP Referenciado FI Referenciado DI CP,"2,55168782","9.607,86926679","23.303,62","24.516,28","272,84","0,00","24.243,44"
2,XPA MM II FIC FIM,"1,48495549","206.015,91600648","231.165,15","305.924,47","2.095,63","0,00","303.828,84"
3,XPA RV FIC FIM,"1,51471586","86.244,99799744","119.600,00","130.636,67","1.655,50","0,00","128.981,17"
4,XPA Vértice Crédito FIM CP,"1,31704969","62.558,82325043","67.664,32","82.393,08","816,74","0,00","81.576,34"


In [16]:
# Cria o DataFrame que conterá o consolidado final
df_result = create_new_df()

# Preenche o consolidado final
for index, row in df.iterrows():

    df_result = df_result.append({
        'CLASSIFICAÇÃO': obtem_classificacao(row['Fundo']),
        'ATIVO': row['Fundo'],
        'EXPOSIÇÃO': "Real",
        'TIT/INST': "-",
        'ATUAL': row["Valor Bruto"],
        'DATA': data_emissao,
        'ON/OFF': "ON",
        'DATA DE VENCIMENTO': "-"
    }, ignore_index=True)

df_result

,CLASSIFICAÇÃO,ATIVO,EXPOSIÇÃO,TIT/INST,ATUAL,DATA,ON/OFF,DATA DE VENCIMENTO
0,Renda Fixa,SulAmérica Inflatie FI Renda Fixa Longo Prazo,Real,-,"23.806,52",11/11/2022,ON,-
1,Renda Fixa,XP Referenciado FI Referenciado DI CP,Real,-,"24.516,28",11/11/2022,ON,-
2,Multimercado,XPA MM II FIC FIM,Real,-,"305.924,47",11/11/2022,ON,-
3,Multimercado,XPA RV FIC FIM,Real,-,"130.636,67",11/11/2022,ON,-
4,Renda Fixa,XPA Vértice Crédito FIM CP,Real,-,"82.393,08",11/11/2022,ON,-


In [18]:
print("Disponibilizando arquivo de consolidado...")

for file in file_objects.keys():

    file_name, file_extension = os.path.splitext(file)

    df_result.to_excel(file_name + ".xlsx", index=False) 
    files.download(file_name + ".xlsx")

Disponibilizando arquivo de consolidado...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>